In [1]:
!pip install xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 27.4 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [2]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=b13554c19782bf8fa33765dc455ea4d6bba124d83545787fa649b3f31f2877f9
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [3]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 8.2 MB 34.7 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=c4158556b41cbb985c11d78cb0a4996345ab1b82720779171563f18c46d4ec80
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [4]:
!pip install tensorflow tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 13.6 MB/s 


In [5]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
import numpy as np
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [7]:
df = pd.read_excel('/content/drive/MyDrive/NLP/reviews.xls')

In [8]:
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [9]:
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), [int(.6*len(df)), int(.8*len(df))])

In [10]:
df_train.head()

,Rating,Content,Date
1457,5,Всё отлично!!!,2017-08-11
19916,5,Мне нравится. Удобно. Еще ни разу не подвел!,2017-06-19
2112,3,Иногда слишком долго приходится ждать пока зап...,2017-08-10
15154,5,Отличное приложение. Легко отслеживать свои ра...,2017-07-22
18329,5,Все грамотно,2017-07-06


In [11]:
df_test.head()

,Rating,Content,Date
12225,3,Всё было хорошо. Редкие проблемы со входом - п...,2017-07-27
1051,1,"Не знаю, что произошло, но приложение стало за...",2017-08-12
14519,5,Замечательное приложение. Решает многие вопрос...,2017-07-23
11410,4,Долго запускается. А так норм.,2017-07-28
9221,1,"Тормозит, не регистрируется ссылаясь на безопа...",2017-08-01


In [12]:
df_val.head()

,Rating,Content,Date
20015,2,После обновления постоянно пишет что прошивка ...,2017-06-17
9828,5,"Хорошее приложение,жаль нет переводов на карты...",2017-07-31
938,1,Было отличное приложение. Потом начал заходить...,2017-08-12
14457,3,Штрафы не оплатить,2017-07-23
395,5,Супер мне нравиться очень и быстро,2017-08-13


In [13]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [14]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [15]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [17]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [18]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [19]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [20]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [21]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [22]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

In [23]:
x_train[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 2], dtype=int32)

In [24]:
import tensorflow as tf
import tensorflow_addons as tfa

In [25]:
import keras

In [26]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [28]:
num_classes = 6
y_train = keras.utils.to_categorical(df_train["Rating"], num_classes)
y_val = keras.utils.to_categorical(df_val["Rating"], num_classes)

In [29]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [30]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
22/22 [==============================] - 6s 235ms/step - loss: 1.3873 - accuracy: 0.6849 - val_loss: 1.0141 - val_accuracy: 0.7202
Epoch 2/20
22/22 [==============================] - 3s 148ms/step - loss: 1.0110 - accuracy: 0.7100 - val_loss: 0.9289 - val_accuracy: 0.7202


In [32]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

9/9 [==============================] - 1s 79ms/step - loss: 0.9686 - accuracy: 0.7016


Test score: 0.9686049222946167
Test accuracy: 0.701597273349762


In [33]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

9/9 [==============================] - 0s 35ms/step


In [34]:
results

array([[0.06182129, 0.10608121, 0.12169348, 0.10083389, 0.13616957,
        0.4734004 ],
       [0.03680671, 0.0759709 , 0.09772222, 0.07120848, 0.11425633,
        0.60403544],
       [0.02106689, 0.05329501, 0.06698974, 0.05103213, 0.0801643 ,
        0.72745186],
       ...,
       [0.0587282 , 0.10340223, 0.11887506, 0.0974602 , 0.13261048,
        0.48892388],
       [0.02678428, 0.06228739, 0.07869924, 0.05928826, 0.09332299,
        0.6796178 ],
       [0.02513865, 0.05981991, 0.07527601, 0.05709981, 0.08947895,
        0.69318664]], dtype=float32)

**Embedding c предобученными векторами с https://rusvectores.org/ru/**

(ruwikiruscorpora_upos_cbow_300_10_2021)

In [35]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=ad1f4860e0f9b9cb803fccd72bab248b2177c03d0b962d9973283feb2627df9a
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [37]:
import sys
import gensim
import zipfile
import wget

In [76]:
!unzip 220.zip

Archive:  220.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [38]:
model_url = 'http://vectors.nlpl.eu/repository/20/220.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    ru_vec_model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [60]:
ru_vec_model.vectors

array([[-5.775997  , -1.4856625 ,  1.4282805 , ..., -0.9964359 ,
        -1.0746299 , -0.8291514 ],
       [ 4.5154796 ,  3.829781  , -0.69469607, ..., -0.43607873,
         4.54931   ,  4.5196757 ],
       [-2.3417003 , -3.3157227 ,  0.34575263, ...,  6.795363  ,
         0.7101504 ,  4.246532  ],
       ...,
       [-0.01564633, -0.05756942,  0.12633698, ..., -0.04677583,
        -0.11077739,  0.01424025],
       [-0.5282987 , -0.03113996, -0.2654572 , ...,  0.00956271,
        -0.18142682,  0.02983635],
       [-0.11416222,  0.9447495 ,  0.25016844, ..., -0.01671841,
         0.48759344, -0.2877875 ]], dtype=float32)

In [62]:
ru_vec_model_keys = [i for i in ru_vec_model.vocab.keys()]

In [72]:
embedding_matrix = [vec[:128] for i, vec in enumerate(ru_vec_model.vectors) if ru_vec_model_keys[i].split('_')[0] in vocabulary]

In [75]:
initializer = tf.keras.initializers.Constant(embedding_matrix[:max_words])

In [84]:
model_rw = Sequential()
model_rw.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer=initializer, input_length=max_len))
model_rw.add(Conv1D(128, 3))
model_rw.add(Activation("relu"))
model_rw.add(GlobalMaxPool1D())
model_rw.add(Dense(10))
model_rw.add(Activation("relu"))
model_rw.add(Dense(num_classes))
model_rw.add(Activation('softmax'))

In [85]:
model_rw.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [86]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model_rw.fit(x_train, y_train,
                       batch_size=batch_size,
                       epochs=epochs,
                       verbose=1,
                       validation_split=0.1,
                       callbacks=[tensorboard, early_stopping])

Epoch 1/20
22/22 [==============================] - 4s 159ms/step - loss: 0.8190 - accuracy: 0.7303 - val_loss: 0.7511 - val_accuracy: 0.7427
Epoch 2/20
22/22 [==============================] - 3s 155ms/step - loss: 0.7120 - accuracy: 0.7571 - val_loss: 0.7188 - val_accuracy: 0.7524


In [87]:
score = model_rw.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

9/9 [==============================] - 0s 36ms/step - loss: 0.7438 - accuracy: 0.7505


Test score: 0.7438122630119324
Test accuracy: 0.750484049320221


**У предобученной модели результат получился лучше**